# Import necessary packages

In [1]:
import copy
import sympy as sp
from sympy.matrices import ones, eye
import re
import dimod
import neal
from dwave.system import DWaveSampler, EmbeddingComposite
from rdkit import Chem

# Input Variables

coords_original - Coordinates considered for distance measure in the hubo<br>torsional_bonds - Torsional bond numbers and their respective coordinate ends<br>coords_rotation_dict - Coordinate numbers and the bonds that affect them<br>distance_pairs - Pairs of coordinates to find distance

In [7]:
mol = Chem.MolFromMolFile('./Minoxidil.mol')
n_atoms = len(mol.GetAtoms())
conformers = mol.GetConformers()
coords_original = {}
for i in range(n_atoms):
    coords_original[i] = list(conformers[0].GetAtomPosition(i))
print(coords_original)

{0: [-3.9714, 0.0468, -0.236], 1: [1.4255, -0.2076, 0.3469], 2: [-0.6244, -1.2585, 0.2517], 3: [-2.5918, 0.0684, -0.0959], 4: [-2.5007, 2.476, -0.3524], 5: [-2.6363, -2.2928, 0.16], 6: [4.1965, 0.1557, -0.3181], 7: [3.3505, -0.9312, -0.9788], 8: [3.3217, 1.324, 0.1334], 9: [2.2046, -1.3715, -0.0708], 10: [2.1762, 0.8481, 1.0239], 11: [0.0815, -0.1536, 0.2026], 12: [-0.5489, 1.196, -0.0111], 13: [-1.8877, 1.2635, -0.1559], 14: [-1.9954, -1.1633, 0.1005]}


In [8]:
torsional_bonds = {
    0: (1, 11),
}

coords_rotation_dict = {}
for i in range(n_atoms):
    coords_rotation_dict[i] = []
coords_rotation_dict[6] = [0]
coords_rotation_dict[7] = [0]
coords_rotation_dict[8] = [0]
coords_rotation_dict[9] = [0]
coords_rotation_dict[10] = [0]

distance_pairs = [
    (0,6), (0,7), (0,8), (0,9), (0,10),
    (2,6), (2,7), (2,8), (2,9), (2,10),
    (3,6), (3,7), (3,8), (3,9), (3,10),
    (4,6), (4,7), (4,8), (4,9), (4,10),
    (5,6), (5,7), (5,8), (5,9), (5,10),
    (11,6), (11,7), (11,8), (11,9), (11,10),
    (12,6), (12,7), (12,8), (12,9), (12,10),
    (13,6), (13,7), (13,8), (13,9), (13,10),
    (14,6), (14,7), (14,8), (14,9), (14,10),
]

In [9]:
coords_dict = copy.deepcopy(coords_original)
final_coords = copy.deepcopy(coords_original)

n_bonds = len(torsional_bonds)  # no. of bonds
n_angles = 8  # no. of discrete angles

x = sp.symbols(f'x(0:{n_bonds*n_angles})')  # hubo variables
print(x)

(x0, x1, x2, x3, x4, x5, x6, x7)


# Function to generate hard constraint

In [10]:
def generate_hard_constraint():
    hard_constraint = 0
    index = 0
    for i in range(n_bonds):
        summation = 0
        for j in range(n_angles):
            summation += x[index]
            index += 1
        hard_constraint += (summation - 1) ** 2
    a_const = sp.Symbol('A_const')
    hard_constraint *= a_const
    return hard_constraint

In [11]:
hard_hubo = generate_hard_constraint()
sp.pprint(hard_hubo)

                                                   2
A_const⋅(x₀ + x₁ + x₂ + x₃ + x₄ + x₅ + x₆ + x₇ - 1) 


# Functions to rotate coordinates with hubo variables

In [12]:
def rotate_coordinates(rotation_matrix, old_coords):
    coord_vector = ones(4, 1)
    coord_vector[0, 0] = old_coords[0]
    coord_vector[1, 0] = old_coords[1]
    coord_vector[2, 0] = old_coords[2]
    coord_rot_vector = sp.expand(rotation_matrix * coord_vector)
    return [coord_rot_vector[0, 0], coord_rot_vector[1, 0], coord_rot_vector[2, 0]]

In [13]:
def generate_thetas():
    angle_incr = 2 * sp.pi / n_angles
    thetas = [i*angle_incr for i in range(n_angles)]
    return thetas

In [14]:
def generate_rotation_matrix(first_coords, second_coords, bond_no):
    x_dash, y_dash, z_dash = first_coords[0], first_coords[1], first_coords[2]
    x_ddash, y_ddash, z_ddash = second_coords[0], second_coords[1], second_coords[2]
    dx = x_ddash - x_dash
    dy = y_ddash - y_dash
    dz = z_ddash - z_dash
    l_sq = dx ** 2 + dy ** 2 + dz ** 2
    l = sp.sqrt(l_sq)
    thetas = generate_thetas()
    c_theta = 0.0
    s_theta = 0.0
    index = n_angles * bond_no
    for i in range(n_angles):
        c_theta += sp.cos(thetas[i]) * x[index]
        s_theta += sp.sin(thetas[i]) * x[index]
        index += 1
    rotation_matrix = eye(4)
    rotation_matrix[0, 0] = (dx ** 2 + (dy ** 2 + dz ** 2) * c_theta) / l_sq
    rotation_matrix[0, 1] = (dx * dy * (1 - c_theta) - dz * l * s_theta) / l_sq
    rotation_matrix[0, 2] = (dx * dz * (1 - c_theta) + dy * l * s_theta) / l_sq
    rotation_matrix[0, 3] = ((x_dash * (dy ** 2 + dz ** 2) - dx * (y_dash * dy + z_dash * dz)) * (1 - c_theta) + (
            y_dash * dz - z_dash * dy) * l * s_theta) / l_sq
    rotation_matrix[1, 0] = (dx * dy * (1 - c_theta) + dz * l * s_theta) / l_sq
    rotation_matrix[1, 1] = (dy ** 2 + (dx ** 2 + dz ** 2) * c_theta) / l_sq
    rotation_matrix[1, 2] = (dy * dz * (1 - c_theta) - dx * l * s_theta) / l_sq
    rotation_matrix[1, 3] = ((y_dash * (dx ** 2 + dz ** 2) - dy * (x_dash * dx + z_dash * dz)) * (1 - c_theta) + (
            z_dash * dx - x_dash * dz) * l * s_theta) / l_sq
    rotation_matrix[2, 0] = (dx * dz * (1 - c_theta) - dy * l * s_theta) / l_sq
    rotation_matrix[2, 1] = (dy * dz * (1 - c_theta) + dx * l * s_theta) / l_sq
    rotation_matrix[2, 2] = (dz ** 2 + (dx ** 2 + dy ** 2) * c_theta) / l_sq
    rotation_matrix[2, 3] = ((z_dash * (dx ** 2 + dy ** 2) - dz * (x_dash * dx + y_dash * dy)) * (1 - c_theta) + (
            x_dash * dy - y_dash * dx) * l * s_theta) / l_sq
    return rotation_matrix

In [15]:
def rotate_all_coordinates():
    for i in coords_original.keys():
        if len(coords_rotation_dict[i]) > 0:
            rot_mat = eye(4, 4)
            for bond_no in coords_rotation_dict[i]:
                temp_rot_mat = generate_rotation_matrix(coords_original[torsional_bonds[bond_no][0]],
                                                        coords_original[torsional_bonds[bond_no][1]], bond_no)
                rot_mat = rot_mat * temp_rot_mat
            coords_dict[i] = rotate_coordinates(rot_mat, coords_original[i])

In [16]:
rotate_all_coordinates()

In [17]:
sp.pprint(coords_original)

{0: [-3.9714, 0.0468, -0.236], 1: [1.4255, -0.2076, 0.3469], 2: [-0.6244, -1.2
585, 0.2517], 3: [-2.5918, 0.0684, -0.0959], 4: [-2.5007, 2.476, -0.3524], 5: 
[-2.6363, -2.2928, 0.16], 6: [4.1965, 0.1557, -0.3181], 7: [3.3505, -0.9312, -
0.9788], 8: [3.3217, 1.324, 0.1334], 9: [2.2046, -1.3715, -0.0708], 10: [2.176
2, 0.8481, 1.0239], 11: [0.0815, -0.1536, 0.2026], 12: [-0.5489, 1.196, -0.011
1], 13: [-1.8877, 1.2635, -0.1559], 14: [-1.9954, -1.1633, 0.1005]}


In [18]:
sp.pprint(coords_dict)

{0: [-3.9714, 0.0468, -0.236], 1: [1.4255, -0.2076, 0.3469], 2: [-0.6244, -1.2
585, 0.2517], 3: [-2.5918, 0.0684, -0.0959], 4: [-2.5007, 2.476, -0.3524], 5: 
[-2.6363, -2.2928, 0.16], 6: [0.120823379484406⋅x₀ + 0.0665153857785361⋅√2⋅x₁ 
+ 0.0122073920726667⋅x₂ - 0.0543079937058695⋅√2⋅x₃ - 0.120823379484406⋅x₄ - 0.
0665153857785361⋅√2⋅x₅ - 0.0122073920726667⋅x₆ + 0.0543079937058695⋅√2⋅x₇ + 4.
07567662051559, 0.469780310645716⋅x₀ - 0.243234086868262⋅√2⋅x₁ - 0.95624848438
2239⋅x₂ - 0.713014397513978⋅√2⋅x₃ - 0.469780310645716⋅x₄ + 0.243234086868262⋅√
2⋅x₅ + 0.956248484382239⋅x₆ + 0.713014397513978⋅√2⋅x₇ - 0.314080310645716, -0.
949539052336607⋅x₀ - 0.710542752441017⋅√2⋅x₁ - 0.471546452545426⋅x₂ + 0.238996
299895591⋅√2⋅x₃ + 0.949539052336607⋅x₄ + 0.710542752441017⋅√2⋅x₅ + 0.471546452
545426⋅x₆ - 0.238996299895591⋅√2⋅x₇ + 0.631439052336607], 7: [0.13676262821957
6⋅x₀ + 0.0033299538845715⋅√2⋅x₁ - 0.130102720450433⋅x₂ - 0.133432674335004⋅√2⋅
x₃ - 0.136762628219576⋅x₄ - 0.0033299538845715⋅√2⋅x₅

# Functions to generate the optimization contraint in hubo

In [19]:
def distance_squared(first_coords, second_coords):
    dis_sq = sp.expand((second_coords[0] - first_coords[0])**2 + (second_coords[1] - first_coords[1])**2 +(second_coords[2] - first_coords[2])**2)
    return dis_sq

def generate_distance_hubo():
    distance_sq = 0
    for pair in distance_pairs:
        distance_sq += distance_squared(coords_dict[pair[0]], coords_dict[pair[1]])
    return distance_sq.expand()

In [20]:
distance_hubo = generate_distance_hubo()
sp.pprint(distance_hubo)

                   2                                                          
87.2943205832167⋅x₀  + 87.2943205832167⋅√2⋅x₀⋅x₁ + 9.56179579958416e-15⋅x₀⋅x₂ 

                                                                              
- 87.2943205832167⋅√2⋅x₀⋅x₃ - 174.588641166433⋅x₀⋅x₄ - 87.2943205832167⋅√2⋅x₀⋅

                                                                              
x₅ - 9.56179579958416e-15⋅x₀⋅x₆ + 87.2943205832167⋅√2⋅x₀⋅x₇ - 2.88506322756245

                         2                                                    
⋅x₀ + 87.2943205832167⋅x₁  + 87.2943205832167⋅√2⋅x₁⋅x₂ + 2.54447071051933e-14⋅

                                                                              
x₁⋅x₃ - 87.2943205832167⋅√2⋅x₁⋅x₄ - 174.588641166433⋅x₁⋅x₅ - 87.2943205832167⋅

                                                                              
√2⋅x₁⋅x₆ - 2.54447071051933e-14⋅x₁⋅x₇ + 0.854158447387186⋅√2⋅x₁ + 87.294320583

       2                                      

# The full hubo expression is written in the file 'full_hubo_expr.txt'

In [21]:
hubo_expr = sp.expand(hard_hubo - distance_hubo)
hubo_expr

A_const*x0**2 + 2*A_const*x0*x1 + 2*A_const*x0*x2 + 2*A_const*x0*x3 + 2*A_const*x0*x4 + 2*A_const*x0*x5 + 2*A_const*x0*x6 + 2*A_const*x0*x7 - 2*A_const*x0 + A_const*x1**2 + 2*A_const*x1*x2 + 2*A_const*x1*x3 + 2*A_const*x1*x4 + 2*A_const*x1*x5 + 2*A_const*x1*x6 + 2*A_const*x1*x7 - 2*A_const*x1 + A_const*x2**2 + 2*A_const*x2*x3 + 2*A_const*x2*x4 + 2*A_const*x2*x5 + 2*A_const*x2*x6 + 2*A_const*x2*x7 - 2*A_const*x2 + A_const*x3**2 + 2*A_const*x3*x4 + 2*A_const*x3*x5 + 2*A_const*x3*x6 + 2*A_const*x3*x7 - 2*A_const*x3 + A_const*x4**2 + 2*A_const*x4*x5 + 2*A_const*x4*x6 + 2*A_const*x4*x7 - 2*A_const*x4 + A_const*x5**2 + 2*A_const*x5*x6 + 2*A_const*x5*x7 - 2*A_const*x5 + A_const*x6**2 + 2*A_const*x6*x7 - 2*A_const*x6 + A_const*x7**2 - 2*A_const*x7 + A_const - 87.2943205832167*x0**2 - 87.2943205832167*sqrt(2)*x0*x1 - 9.56179579958416e-15*x0*x2 + 87.2943205832167*sqrt(2)*x0*x3 + 174.588641166433*x0*x4 + 87.2943205832167*sqrt(2)*x0*x5 + 9.56179579958416e-15*x0*x6 - 87.2943205832167*sqrt(2)*x0*x7 

In [22]:
def hubo_expr_to_dict():
    hubo_args = hubo_expr.args
    hubo_dict = {}
    for monom in hubo_args:
        dict_value = monom.as_coeff_mul()[0]
        monom_key = []
        monom_coeffs = monom.as_coeff_mul()[1]
        for monom_item in monom_coeffs:
            if re.match("^x(\d*)\d$", str(monom_item)):
                monom_key.append(int(str(monom_item)[1:]))
            elif re.match("^x(\d*)\\*\\*(\d)$", str(monom_item)):
                monom_key.append(int(str(monom_item).split('**')[0][1:]))
            else:
                dict_value *= monom_item
        if len(monom_key) > 0:
            monom_key.sort()
            dict_key = tuple(monom_key)
        else:
            dict_key = ()
        dict_value = dict_value.evalf()
        if dict_key in hubo_dict:
            hubo_dict[dict_key] += dict_value
        else:
            hubo_dict[dict_key] = dict_value
    return hubo_dict

In [23]:
hubo_dict = hubo_expr_to_dict()
print(hubo_dict)

{(): A_const - 1244.02674465435, (0,): -1.0*A_const - 84.4092573556542, (6,): -1.0*A_const - 82.7009404608799, (4,): -1.0*A_const - 90.1793838107791, (2,): -1.0*A_const - 91.8877007055535, (3,): -1.0*A_const - 92.5823785886499, (7,): -1.0*A_const - 82.0062625777835, (1,): -1.0*A_const - 88.5022830439272, (5,): -1.0*A_const - 86.0863581225062, (0, 6): 2.0*A_const + 9.56179579958416e-15, (2, 4): 2.0*A_const + 9.56179579958416e-15, (1, 7): 2.0*A_const + 2.54447071051933e-14, (3, 5): 2.0*A_const + 2.54447071051933e-14, (2, 6): 2.0*A_const + 174.588641166433, (3, 7): 2.0*A_const + 174.588641166433, (0, 4): 2.0*A_const + 174.588641166433, (1, 5): 2.0*A_const + 174.588641166433, (0, 2): 2.0*A_const - 9.56179579958416e-15, (4, 6): 2.0*A_const - 9.56179579958416e-15, (1, 3): 2.0*A_const - 2.54447071051933e-14, (5, 7): 2.0*A_const - 2.54447071051933e-14, (0, 1): 2.0*A_const - 123.45281208693, (0, 3): 2.0*A_const + 123.45281208693, (0, 5): 2.0*A_const + 123.45281208693, (0, 7): 2.0*A_const - 123.

In [24]:
file_name = "hubo_dict.txt"
f = open(file_name, "w")
print('full hubo dictionary written in file - hubo_dict.txt')
f.write(str(hubo_dict))
f.close()

full hubo dictionary written in file - hubo_dict.txt


In [25]:
#This is used to check the maximum coefficient appearing in Hubo_B
A_const=0
read_dictionary_B = open(file_name, 'r').read()
HUBO_B=eval(read_dictionary_B)

#We set the Hard constraint strength as the (maximum coefficient appearing in Hubo_B)*const.
#const was empirically selected to be 10
const=10
A_const=max(map(abs, list(HUBO_B.values())))*const
# A_const = 1000

#read the final HUBO
read_dictionary= open(file_name, 'r').read()
HUBO=eval(read_dictionary)

In [26]:
print("Current size of the HUBO:",len(HUBO)) 

Current size of the HUBO: 37


In [27]:
def threshold_approx(h, val=1):
    d =h.copy()
    monoms = h.keys()
    for m in monoms:     
        temp = d[m]
        if (temp < 0.0):
            temp = -1.0 * temp
        if (temp <= (10.0 ** (val))):
            del d[m]
    return d

In [28]:
#Coefficints with absolute value less than 10^{threshold} are deleted from the HUBO.
threshold=2

HUBO=threshold_approx(HUBO,threshold)

print("Size of the HUBO after threshold approximation:",len(HUBO))

Size of the HUBO after threshold approximation: 37


In [29]:
#calculate the strength parameter needed by make_quadratic
max_hubo_value=max(map(abs, list(HUBO.values())))
strength=1.5*max_hubo_value
#generate the bqm
bqm = dimod.make_quadratic(HUBO, strength, dimod.BINARY)

In [40]:
def display_soln_angles(solution):
    thetas = generate_thetas()
    for key, value in solution.items():
        if solution[key] == 1:
            print('bond ', (key//n_angles+1), ': ', thetas[key%n_angles])

In [45]:
sampler = neal.SimulatedAnnealingSampler()
sample_size=10
sampleset = sampler.sample(bqm, num_reads=sample_size)
print(sampleset)

   0  1  2  3  4  5  6  7       energy num_oc.
6  0  0  0  1  0  0  0  0 -1336.609123       1
4  0  1  0  0  0  0  0  0 -1332.529028       1
0  0  0  0  0  0  1  0  0 -1330.113103       1
5  0  0  0  0  0  1  0  0 -1330.113103       1
9  0  0  0  0  0  1  0  0 -1330.113103       1
8  0  0  0  0  0  0  1  0 -1326.727685       1
1  0  0  0  0  0  0  0  1 -1326.033007       1
2  0  0  0  0  0  0  0  1 -1326.033007       1
3  0  0  0  0  0  0  0  1 -1326.033007       1
7  0  0  0  0  0  0  0  1 -1326.033007       1
['BINARY', 10 rows, 10 samples, 8 variables]


In [43]:
sa_soln = sampleset.first.sample
print(sa_soln)
display_soln_angles(sa_soln)

{0: 0, 1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0}
bond  1 :  3*pi/4


In [46]:
sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample(bqm, num_reads=1000)
print(sampleset.slice(10))

   0  1  2  3  4  5  6  7       energy num_oc. chain_.
0  0  0  0  1  0  0  0  0 -1336.609123      41     0.0
1  0  0  1  0  0  0  0  0 -1335.914445      94     0.0
2  0  0  0  0  1  0  0  0 -1334.206128      92     0.0
3  0  1  0  0  0  0  0  0 -1332.529028     143     0.0
4  0  0  0  0  0  1  0  0 -1330.113103     173     0.0
5  1  0  0  0  0  0  0  0 -1328.436002      88     0.0
6  0  0  0  0  0  0  1  0 -1326.727685      61     0.0
7  0  0  0  0  0  0  0  1 -1326.033007      84     0.0
8  0  0  1  1  0  0  0  0 10888.317811       4     0.0
9  0  0  0  1  1  0  0  0 10890.026127       1     0.0
['BINARY', 10 rows, 781 samples, 8 variables]


In [34]:
qa_soln = sampleset.first.sample
print(qa_soln)
display_soln_angles(qa_soln)

{0: 0, 1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0}
bond  1 :  3*pi/4
